# Data generation: using python to sweep over methods and parameters

In this notebook, we illustrate how to use python to generate and run a list of commands. In this example, we generate a list of QIIME 1.9.0 ``assign_taxonomy.py`` commands, though this workflow for command generation is generally very useful for performing *parameter sweeps* (i.e., exploration of sets of parameters for achieving a specific result for comparative purposes). 

## Environment preparation

In [11]:
from os.path import join, expandvars 
from joblib import Parallel, delayed
from glob import glob
from tax_credit.framework_functions import (parameter_sweep,
                                            generate_per_method_biom_tables,
                                            move_results_to_repository)

In [12]:
project_dir = expandvars("$HOME/Desktop/projects/short-read-tax-assignment")
analysis_name= "mock-community"
data_dir = join(project_dir, "data", analysis_name)

reference_database_dir = expandvars("$HOME/Desktop/projects/short-read-tax-assignment/data/ref_dbs/")
results_dir = expandvars("$HOME/Desktop/projects/mock-community/")

## Preparing data set sweep

First, we're going to define the data sets that we'll sweep over. The following cell does not need to be modified unless if you wish to change the datasets or reference databases used in the sweep. Here we will use a single mock community, but two different versions of the reference database.

In [32]:
dataset_reference_combinations = [
 ('mock-3', 'gg_13_8_otus_trim150'),
 ('mock-3', 'gg_13_8_otus_full16S'),
 ('mock-9', 'unite_20.11.2016_trim100'),
 ('mock-9', 'unite_20.11.2016_fullITS'),
]

reference_dbs = {'gg_13_8_otus_trim150' : (join(reference_database_dir, 'gg_13_8_otus/99_otus_clean_515f-806r_trim150.fasta'), 
                                   join(reference_database_dir, 'gg_13_8_otus/99_otu_taxonomy_clean.tsv')),
                 'gg_13_8_otus_full16S' : (join(reference_database_dir, 'gg_13_8_otus/99_otus_clean.fasta'), 
                                   join(reference_database_dir, 'gg_13_8_otus/99_otu_taxonomy_clean.tsv')),
                 'unite_20.11.2016_trim100' : (join(reference_database_dir, 'unite_20.11.2016/sh_refs_qiime_ver7_99_20.11.2016_dev_clean_ITS1Ff-ITS2r_trim100.fasta'), 
                                   join(reference_database_dir, 'unite_20.11.2016/sh_taxonomy_qiime_ver7_99_20.11.2016_dev_clean.tsv')),
                 'unite_20.11.2016_fullITS' : (join(reference_database_dir, 'unite_20.11.2016/sh_refs_qiime_ver7_99_20.11.2016_dev_clean.fasta'), 
                                   join(reference_database_dir, 'unite_20.11.2016/sh_taxonomy_qiime_ver7_99_20.11.2016_dev_clean.tsv'))
                }

## Preparing the method/parameter combinations and generating commands

Now we set the methods and method-specific parameters that we want to sweep. Modify to sweep other methods. Note how method_parameters_combinations feeds method/parameter combinations to parameter_sweep() in the cell below.

### Assignment Using QIIME 1 or Command-Line Classifiers

Here we provide an example of taxonomy assignment using legacy ``QIIME 1`` classifiers executed on the command line. To accomplish this, we must first convert ``commands`` to a string, which we then pass to bash for execution. As ``QIIME 1`` is written in python-2, we must also activate a separate environment in which QIIME 1 [has been installed](http://qiime.org/install/install.html). If any environmental variables need to be set (in this example, the [RDP_JAR_PATH](http://qiime.org/install/alternative.html#rdp-jar-path-environment-variable)), we must also source the .bashrc file.

In [33]:

method_parameters_combinations = { # probabalistic classifiers
              'rdp': {'confidence': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5,
                                     0.6, 0.7, 0.8, 0.9, 1.0]},
             }

Now enter the template of the command to sweep, and generate a list of commands with ``parameter_sweep()``.

Fields must adhere to following format:

                      {0} = output directory
                      {1} = input data
                      {2} = reference sequences
                      {3} = reference taxonomy
                      {4} = method name
                      {5} = other parameters

In [34]:
command_template = "mkdir -p {0} ; assign_taxonomy.py -v -i {1} -o {0} -r {2} -t {3} -m {4} {5} --rdp_max_memory 16000"
        
commands = parameter_sweep(data_dir, results_dir, reference_dbs,
                           dataset_reference_combinations,
                           method_parameters_combinations, command_template,
                           infile='rep_seqs.fna',)


As a sanity check, we can look at the first command that was generated and the number of commands generated.

In [35]:
print(len(commands))
commands[0]

11


'mkdir -p /Users/nbokulich/Desktop/projects/mock-community/mock-9/unite_20.11.2016_trim100/rdp/0.0 ; assign_taxonomy.py -v -i /Users/nbokulich/Desktop/projects/short-read-tax-assignment/data/mock-community/mock-9/rep_seqs.fna -o /Users/nbokulich/Desktop/projects/mock-community/mock-9/unite_20.11.2016_trim100/rdp/0.0 -r /Users/nbokulich/Desktop/projects/short-read-tax-assignment/data/ref_dbs/unite_20.11.2016/sh_refs_qiime_ver7_99_20.11.2016_dev_clean_ITS1Ff-ITS2r_trim100.fasta -t /Users/nbokulich/Desktop/projects/short-read-tax-assignment/data/ref_dbs/unite_20.11.2016/sh_taxonomy_qiime_ver7_99_20.11.2016_dev_clean.tsv -m rdp --confidence 0.0 --rdp_max_memory 16000'

Let's use a simple subroutine function to run CLI commands in parallel. You will not need to modify the values in this cell unless if the conda environment in which qiime1 is installed has a different name, or if environmental parameters are not stored in .bashrc.

In [36]:
def run_cli(command, env='qiime1'):
    ! source activate qiime1; source ~/.bashrc; {command}

Finally, we run our commands.

In [37]:
Parallel(n_jobs=4)(delayed(run_cli)(command) for command in commands)

[None, None, None, None, None, None, None, None, None, None, None]

## Generate per-method biom tables

Modify the taxonomy_glob below to point to the taxonomy assignments that were generated above. This may be necessary if filepaths were altered in the preceding cells.

In [23]:
taxonomy_glob = join(results_dir, '*', '*', '*', '*', 'rep_seqs_tax_assignments.txt')
generate_per_method_biom_tables(taxonomy_glob, data_dir)

## Move result files to repository

Add results to the short-read-taxa-assignment directory (e.g., to push these results to the repository or compare with other precomputed results in downstream analysis steps). The precomputed_results_dir path and methods_dirs glob below should not need to be changed unless if substantial changes were made to filepaths in the preceding cells.

In [26]:
precomputed_results_dir = join(project_dir, "data", "precomputed-results", analysis_name)
for community in dataset_reference_combinations:
    method_dirs = glob(join(results_dir, community[0], '*', '*', '*'))
    move_results_to_repository(method_dirs, precomputed_results_dir)

Do not forget to copy the expected taxonomy files for this mock community!

In [27]:
for community in dataset_reference_combinations:
    community_dir = join(precomputed_results_dir, community[0])
    exp_observations = join(community_dir, '*', 'expected')
    new_community_exp_dir = join(community_dir, community[1], 'expected')
    !mkdir {new_community_exp_dir}; cp {exp_observations}/* {new_community_exp_dir}